In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
HEIGHT = 48
WIDTH = 24

In [3]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-
from __future__ import division, print_function, unicode_literals
import tensorflow as tf
from time import time
from keras.utils import np_utils#one hot
import os
from skimage import data
from skimage import transform
import skimage
import numpy as np

# Download the dataset


def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                  if os.path.isdir(os.path.join(data_directory,d))]
    labels=[]
    images=[]
    for d in directories:
        label_directory = os.path.join(data_directory,d)
        file_names = [os.path.join(label_directory,f)
                     for f in os.listdir(label_directory)
                     if f.endswith('.jpg')]
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(str(d))
    return images, labels

Using TensorFlow backend.


In [4]:
ROOT_PATH = '/home/tang/targa/exp3/'
train_data_directory = os.path.join(ROOT_PATH, "padding48_24/train")
#validation_data_directory = os.path.join(ROOT_PATH, "16_8_1568_train") 16_8_4448_train  16_8_1248_train 

test_data_directory = os.path.join(ROOT_PATH, "padding48_24/test")

In [5]:
train_images,train_labels = load_data(train_data_directory)
test_images,test_labels = load_data(test_data_directory)


In [6]:
#encoding trY teY
from numpy import array
#from numpy import argmax
#from keras.utils import to_categorical
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example

#train_labels = np_utils.to_categorical(train_labels,num_classes = None)
#test_labels = np_utils.to_categorical(test_labels,num_classes = None)
trL,teL=array(train_labels),array(test_labels)
label_encoder = LabelEncoder()

In [7]:
integer_encoded_trL = label_encoder.fit_transform(trL)
integer_encoded_teL = label_encoder.fit_transform(teL)


In [8]:
trainLabels = np_utils.to_categorical(integer_encoded_trL,num_classes = 32)
testLabels = np_utils.to_categorical(integer_encoded_teL,num_classes = 32)


In [9]:
########convert rgb images into black and white
#convert images into size of vector
from skimage.color import rgb2gray

trainImages = rgb2gray(np.array(train_images))
testImages = rgb2gray(np.array(test_images))

train_flat_images = np.reshape(trainImages,(len(trainImages),HEIGHT*WIDTH))
test_flat_images = np.reshape(testImages,(len(testImages),HEIGHT*WIDTH))


In [10]:
train_flat_images[0].shape

(1152,)

In [11]:
print('train images shape:\t\t{}'.format(train_flat_images.shape))

print('test images shape:\t\t{}'.format(test_flat_images.shape))



train images shape:		(1246, 1152)
test images shape:		(411, 1152)


In [12]:
batch_size = 100
n_batch = len(trainLabels)//batch_size
lr = tf.Variable(0.001,dtype = tf.float32)

In [13]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean) # 平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histgram',var) # 直方图

In [14]:

#命名空间
with tf.name_scope('input'):
    #定义两个placeholder 
    #把批次的图片传进去，开始是100张一次的传送，直到最后
    x = tf.placeholder(tf.float32,[None,WIDTH*HEIGHT],name='x-input')
    y = tf.placeholder(tf.float32,[None,32],name='y-input')
    #定义一个dropout的值
    keep_prob = tf.placeholder(tf.float32,name = 'non-dropout-rate')
with tf.name_scope('layer'):
    with tf.name_scope('W1'):
        #第一层
        W1 = tf.Variable(tf.truncated_normal([WIDTH*HEIGHT,50],stddev=0.1),name='W1')#改成截断正太分布
    with tf.name_scope('b1'):
        
        b1 = tf.Variable(tf.zeros([50])+0.1,name = 'b1')
    with tf.name_scope('out1'):
        
        output1 = tf.matmul(x,W1) + b1
    with tf.name_scope('L1'):
        L1 = tf.nn.tanh(output1)
    with tf.name_scope('L1-drop'):
        #L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
        L1_drop = tf.nn.dropout(L1,keep_prob)

    '''
    with tf.name_scope('W2'):
        #第二层
        W2 = tf.Variable(tf.truncated_normal([1000,2000],stddev=0.1),name='W2')
    with tf.name_scope('b2'):
        b2 = tf.Variable(tf.zeros([2000])+0.2,name='b2')
    with tf.name_scope('output2'):
        output2 = tf.matmul(L1_drop,W2)+b2
    with tf.name_scope('L2'):
        
        L2 = tf.nn.tanh(output2)
    with tf.name_scope('L2_drop'):
        #L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
        L2_drop = tf.nn.dropout(L2,keep_prob)

    with tf.name_scope('W3'):
        #第三层
        W3 = tf.Variable(tf.truncated_normal([2000,1000],stddev=0.1),name='W3')
    with tf.name_scope('b3'):
        b3 = tf.Variable(tf.zeros([1000])+0.1,name='b3')
    with tf.name_scope('output3'):   
        output3 = tf.matmul(L2_drop,W3)+b3
    with tf.name_scope('L3'):
        L3 = tf.nn.tanh(output3)
    with tf.name_scope('L3-drop'):
        #L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
        L3_drop = tf.nn.dropout(L3,keep_prob)
        '''
    with tf.name_scope('W4'):
    #第四层
        W4 = tf.Variable(tf.truncated_normal([50,32],stddev=0.1))
    with tf.name_scope('b4'):        
        b4 = tf.Variable(tf.zeros([32])+0.1)
    with tf.name_scope('output4'):
        output4 = tf.matmul(L1_drop,W4)+ b4
    with tf.name_scope('softmax'):
        #预测分类
        prediction = tf.nn.softmax(output4)
with tf.name_scope('loss'):
    #定义一个损失函数
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits = prediction))#联合交叉熵
    tf.summary.scalar('cross_entropy',loss)
    #loss = tf.reduce_mean(tf.square(y-prediction))
with tf.name_scope('train'):
    #定义一个方法来收敛神经网络，使用梯度下降法,学习率=0.1
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    #train_step = tf.train.AdamOptimizer(0.1).minimize(loss)
#初始化
init = tf.global_variables_initializer()
with tf.name_scope('accuracy'):
    with tf.name_scope('correct-prediction'):
        #定义一个求准确率的方法
        #如果有两个相同的就会返回TRUE，不相同就返回false，然后就依次存入correct_prediction
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中的最大值
    with tf.name_scope('accuracy'):    
        #求准确率
        #首先把bool值转化成32位的浮点数，然后求平均值
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [15]:
merged = tf.summary.merge_all()

In [16]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [18]:
with tf.Session() as sess:
    sess.run(init)
    #路径和文件名
    train_writer = tf.summary.FileWriter('logs/padding48*24targa/train',sess.graph)
   
    test_writer = tf.summary.FileWriter('logs/padding48*24targa/test',sess.graph)
    for epoch in range(200001):#把所有的图片训练20次
        #一批一批的迭代图片，一次迭代100张，一共运行n_batch次
        for batch in range(n_batch):
            #每次传入一百张图片
            batch_xs,batch_ys = next_batch(batch_size,train_flat_images,trainLabels)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        train_batch_xs,train_batch_ys = train_flat_images,trainLabels
        summary = sess.run(merged,feed_dict={x:train_batch_xs,y:train_batch_ys,keep_prob:1.0})           
        train_writer.add_summary(summary,epoch)
            
        test_batch_xs,test_batch_ys = test_flat_images,testLabels
        summary = sess.run(merged,feed_dict={x:test_batch_xs,y:test_batch_ys,keep_prob:1.0})
        test_writer.add_summary(summary,epoch)  
        
        test_acc = sess.run(accuracy,feed_dict={x:test_flat_images,y:testLabels,keep_prob:1.0})        
        train_acc = sess.run(accuracy,feed_dict={x:train_flat_images,y:trainLabels,keep_prob:1.0})
        test_loss = sess.run(loss,feed_dict={x:test_flat_images,y:testLabels,keep_prob:1.0}) 
        train_loss = sess.run(loss,feed_dict={x:train_flat_images,y:trainLabels,keep_prob:1.0})
        if epoch%2000 == 0:
            
            print('epoch=' + str(epoch) + ',train accuracy='+ str(train_acc)+',test accuracy=' + str(test_acc)+',train loss='+str(train_loss)+',test loss='+str(test_loss))

epoch=0,train accuracy=0.033707865,test accuracy=0.02919708,train loss=3.4655209,test loss=3.4657373
epoch=2000,train accuracy=0.56982344,test accuracy=0.40389293,train loss=2.9610088,test loss=3.1260285
epoch=4000,train accuracy=0.58266455,test accuracy=0.40632603,train loss=2.9381683,test loss=3.1192493
epoch=6000,train accuracy=0.58747995,test accuracy=0.40632603,train loss=2.9309795,test loss=3.115638
epoch=8000,train accuracy=0.58908504,test accuracy=0.40632603,train loss=2.928043,test loss=3.1139941
epoch=10000,train accuracy=0.59229535,test accuracy=0.40145984,train loss=2.9240286,test loss=3.1139023
epoch=12000,train accuracy=0.6139647,test accuracy=0.4136253,train loss=2.9022934,test loss=3.1043806
epoch=14000,train accuracy=0.6388443,test accuracy=0.43552312,train loss=2.8778207,test loss=3.0866182
epoch=16000,train accuracy=0.64686996,test accuracy=0.43552312,train loss=2.8701208,test loss=3.0825653
epoch=18000,train accuracy=0.6484751,test accuracy=0.4379562,train loss=2.86

epoch=160000,train accuracy=0.70545745,test accuracy=0.459854,train loss=2.8094254,test loss=3.0591633
epoch=162000,train accuracy=0.70545745,test accuracy=0.45742092,train loss=2.8093972,test loss=3.057481
epoch=164000,train accuracy=0.70545745,test accuracy=0.459854,train loss=2.8093722,test loss=3.0595732
epoch=166000,train accuracy=0.70545745,test accuracy=0.45742092,train loss=2.809319,test loss=3.0594678
epoch=168000,train accuracy=0.70545745,test accuracy=0.45498782,train loss=2.8093245,test loss=3.0604022
epoch=170000,train accuracy=0.70626,test accuracy=0.4647202,train loss=2.808552,test loss=3.0566456
epoch=172000,train accuracy=0.70626,test accuracy=0.459854,train loss=2.808547,test loss=3.0559156
epoch=174000,train accuracy=0.70626,test accuracy=0.4622871,train loss=2.8084984,test loss=3.0565283
epoch=176000,train accuracy=0.70626,test accuracy=0.4622871,train loss=2.8085115,test loss=3.0557966
epoch=178000,train accuracy=0.70626,test accuracy=0.4647202,train loss=2.8085234